<a href="https://colab.research.google.com/github/huiqin97/Deep-Learning-in-Learning-Analysis/blob/main/FYP_LSTM_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
stInfo  = pd.read_csv('https://raw.githubusercontent.com/huiqin97/Deep-Learning-in-Learning-Analysis/main/studentInfo.csv')
stVLE  = pd.read_csv('/content/drive/MyDrive/studentVle.csv')
VLE  = pd.read_csv('https://raw.githubusercontent.com/huiqin97/Deep-Learning-in-Learning-Analysis/main/vle.csv')
# df_assessment = pd.read_csv('https://raw.githubusercontent.com/huiqin97/Deep-Learning-in-Learning-Analysis/main/assessments.csv')
stAss  = pd.read_csv('https://raw.githubusercontent.com/huiqin97/Deep-Learning-in-Learning-Analysis/main/studentAssessment.csv')
# df_course_details = pd.read_csv('https://raw.githubusercontent.com/huiqin97/Deep-Learning-in-Learning-Analysis/main/courses.csv')
stReg  =  pd.read_csv('https://raw.githubusercontent.com/huiqin97/Deep-Learning-in-Learning-Analysis/main/studentRegistration.csv')

Check the students' final result

In [3]:
stInfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   code_module           32593 non-null  object
 1   code_presentation     32593 non-null  object
 2   id_student            32593 non-null  int64 
 3   gender                32593 non-null  object
 4   region                32593 non-null  object
 5   highest_education     32593 non-null  object
 6   imd_band              31482 non-null  object
 7   age_band              32593 non-null  object
 8   num_of_prev_attempts  32593 non-null  int64 
 9   studied_credits       32593 non-null  int64 
 10  disability            32593 non-null  object
 11  final_result          32593 non-null  object
dtypes: int64(3), object(9)
memory usage: 3.0+ MB


In [4]:
stInfo_update = stInfo[(stInfo['final_result'] =='Pass') | (stInfo['final_result'] =='Distinction')
                       | (stInfo['final_result'] =='Fail') ] 

In [5]:
stInfo_update.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22437 entries, 0 to 32592
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   code_module           22437 non-null  object
 1   code_presentation     22437 non-null  object
 2   id_student            22437 non-null  int64 
 3   gender                22437 non-null  object
 4   region                22437 non-null  object
 5   highest_education     22437 non-null  object
 6   imd_band              21562 non-null  object
 7   age_band              22437 non-null  object
 8   num_of_prev_attempts  22437 non-null  int64 
 9   studied_credits       22437 non-null  int64 
 10  disability            22437 non-null  object
 11  final_result          22437 non-null  object
dtypes: int64(3), object(9)
memory usage: 2.2+ MB


In [6]:
stVLE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10655280 entries, 0 to 10655279
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   code_module        object
 1   code_presentation  object
 2   id_student         int64 
 3   id_site            int64 
 4   date               int64 
 5   sum_click          int64 
dtypes: int64(4), object(2)
memory usage: 487.8+ MB


In [7]:
stVLE_update = stVLE[stVLE['date']>=0]
stVLE_update.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9966292 entries, 10847 to 10655279
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   code_module        object
 1   code_presentation  object
 2   id_student         int64 
 3   id_site            int64 
 4   date               int64 
 5   sum_click          int64 
dtypes: int64(4), object(2)
memory usage: 532.3+ MB


Create dictionary for VLE activity

In [12]:
courses = stInfo_update['code_module'].unique()
print(courses)


['AAA' 'BBB' 'CCC' 'DDD' 'EEE' 'FFF' 'GGG']


In [9]:
dic_site_acitivity = {v:k for v, k in zip(VLE['id_site'], VLE['activity_type'])}
actions_len = len(VLE['activity_type'].unique())

actions_dic = VLE['activity_type'].unique().tolist()

actions_dic = {v: k for k, v in enumerate(actions_dic)}
dic_acitivity_idx = {v:k for v, k in actions_dic.items()}

print(dic_site_acitivity)
print('\n')
print(dic_acitivity_idx)

{546943: 'resource', 546712: 'oucontent', 546998: 'resource', 546888: 'url', 547035: 'resource', 546614: 'homepage', 546897: 'url', 546678: 'oucontent', 546933: 'resource', 546708: 'oucontent', 546995: 'resource', 546884: 'url', 547031: 'resource', 546891: 'url', 546675: 'oucontent', 546927: 'resource', 546705: 'oucontent', 546991: 'resource', 546876: 'subpage', 547027: 'resource', 546671: 'oucontent', 546923: 'resource', 546702: 'oucontent', 546988: 'resource', 546872: 'subpage', 547024: 'resource', 546668: 'oucontent', 546920: 'resource', 546699: 'oucontent', 546985: 'resource', 546739: 'oucontent', 547021: 'resource', 546665: 'oucontent', 546917: 'resource', 546695: 'oucontent', 546982: 'resource', 546736: 'oucontent', 547018: 'resource', 546661: 'glossary', 546692: 'oucontent', 546972: 'resource', 546733: 'oucontent', 547014: 'resource', 546658: 'forumng', 546914: 'resource', 546689: 'oucontent', 546968: 'resource', 546728: 'oucontent', 547011: 'resource', 547066: 'oucollaborate', 

In [10]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

labelencoder.fit(["Distinction", "Fail", "Pass"])

LabelEncoder()

Transform the data into training data

In [15]:
# extract features for the course  BBB, 2013 as the training and 2014 as the test

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

labelencoder.fit(["Distinction", "Fail", "Pass"])

courses = ['DDD'] # we can also add AAA, BBB, CCC

for i, course in enumerate(courses):
    
    stVLE_subset = stVLE_update[stVLE_update['code_module'] == course]
    stInfo_update_filtered = stInfo_update[stInfo_update['code_module'] == course]
    
    # filter multiple attemps
    stInfo_update_filtered.drop_duplicates(subset=['id_student'], keep=False, inplace=True)
    
    stVLE_subset_filtered = stVLE_subset[stVLE_subset['id_student'].isin(stInfo_update_filtered['id_student'])]
    
    stInfo_update_filtered = stInfo_update_filtered[stInfo_update_filtered['id_student'].isin(stVLE_subset_filtered['id_student'])]
    
    
    stVLE_subset_train = stVLE_subset_filtered[(stVLE_subset_filtered['code_presentation'] == '2013B') | 
                                              (stVLE_subset_filtered['code_presentation'] == '2013J')]

    stVLE_subset_test = stVLE_subset_filtered[(stVLE_subset_filtered['code_presentation'] == '2014B') | 
                                              (stVLE_subset_filtered['code_presentation'] == '2014J')]
  
    stinfo_train = stInfo_update_filtered[stInfo_update_filtered['id_student'].isin(stVLE_subset_train['id_student'])]

    stinfo_test = stInfo_update_filtered[stInfo_update_filtered['id_student'].isin(stVLE_subset_test['id_student'])]

    
    studentno_train = len(stVLE_subset_train['id_student'].unique())
    
     
    stinfo_train['arrayIdx'] = np.arange(studentno_train)
    
     
    print('the number of students {} in train course {}'.format(studentno_train, course))
    
    cc = stVLE_subset_train.replace({'id_site': dic_site_acitivity}, inplace=False) 
    stVLE_subset_train['activity_type'] = cc['id_site']
    
    cc = stVLE_subset_train.replace({'activity_type': dic_acitivity_idx}, inplace=False) 
    stVLE_subset_train['activity_idx'] = cc['activity_type']
   
    
    stVLE_subset_train = pd.merge(stVLE_subset_train, stinfo_train[{'arrayIdx', 'id_student'}], on='id_student', validate='m:1')
    
    total_days = np.max(stVLE_subset_train['date'].values) + 1
    
    actionArray_train = np.zeros([studentno_train, total_days, actions_len])
    
    y_train = np.zeros(studentno_train)
         
    y_train = labelencoder.transform(stinfo_train['final_result'])
   

    for day in range(total_days):
        
        #print('running the course {} and the day : {}'.format(course, day))
        
        stVLE_days = stVLE_subset_train[stVLE_subset_train['date'] == day]
    
        yy = stVLE_days.groupby(['arrayIdx', 'activity_idx'])['sum_click'].sum().unstack(fill_value=0)

        newyy = yy.reindex(np.arange(actions_len), axis=1, fill_value=0)

        newzz = newyy.reindex(np.arange(studentno_train), axis=0, fill_value=0)

        newzz = newzz.values

        actionArray_train[:,day,:] = newzz
        
    ####
    
    studentno_test = len(stVLE_subset_test['id_student'].unique())
    
     
    stinfo_test['arrayIdx'] = np.arange(studentno_test)
    
     
    print('the number of students {} in test course {}'.format(studentno_test, course))
    
    cc = stVLE_subset_test.replace({'id_site': dic_site_acitivity}, inplace=False) 
    stVLE_subset_test['activity_type'] = cc['id_site']
    
    cc = stVLE_subset_test.replace({'activity_type': dic_acitivity_idx}, inplace=False) 
    stVLE_subset_test['activity_idx'] = cc['activity_type']
   
    
    stVLE_subset_test = pd.merge(stVLE_subset_test, stinfo_test[{'arrayIdx', 'id_student'}], on='id_student', validate='m:1')
    
    total_days = np.max(stVLE_subset_test['date'].values) + 1
    
    actionArray_test= np.zeros([studentno_test, total_days, actions_len])
    
    y_test = np.zeros(studentno_test)
         
    y_test = labelencoder.transform(stinfo_test['final_result'])
   
    
    for day in range(total_days):
        
        #print('running the course {} and the day : {}'.format(course, day))
        
        stVLE_days = stVLE_subset_test[stVLE_subset_test['date'] == day]
    
        yy = stVLE_days.groupby(['arrayIdx', 'activity_idx'])['sum_click'].sum().unstack(fill_value=0)

        newyy = yy.reindex(np.arange(actions_len), axis=1, fill_value=0)

        newzz = newyy.reindex(np.arange(studentno_test), axis=0, fill_value=0)

        newzz = newzz.values

        actionArray_test[:,day,:] = newzz

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


the number of students 2172 in train course DDD


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

the number of students 1850 in test course DDD


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [16]:
# merge distinction and pass 
y_train[y_train==2] = 0 
y_test[y_test==2] = 0

In [17]:
[a, b] = np.unique(y_train, return_counts=True)
print(a)
print(b)

[a, b] = np.unique(y_test, return_counts=True)
print(a)
print(b)

[0 1]
[1382  790]
[0 1]
[1264  586]


In [20]:
# construct the data form for LSTM

total_weeks = 37

usedweeks = 35 # use 35-weeks' data

X = list()

for i in range(total_weeks):
    seq_x = np.sum(actionArray_train[:, (7*i):(7*i+6), :], axis=1)
    X.append(seq_x)
X = np.array(X)


actionWeeks = np.swapaxes(X, 0, 1)

def prepare_data(actionWeeks, weeks=1):
    return actionWeeks[:,:weeks,:]

X_train = prepare_data(actionWeeks, weeks=usedweeks)


In [21]:
X = list()

for i in range(total_weeks):
    seq_x = np.sum(actionArray_test[:, (7*i):(7*i+6), :], axis=1)
    X.append(seq_x)
X = np.array(X)

actionWeeks = np.swapaxes(X, 0, 1)

def prepare_data(actionWeeks, weeks=1):
    return actionWeeks[:,:weeks,:]

X_test = prepare_data(actionWeeks, weeks=usedweeks)

from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

XX = np.reshape(X_train, (X_train.shape[0]*X_train.shape[1], X_train.shape[2]))
scaler.fit(np.log(XX + 1))
XX = scaler.transform(np.log(XX+1))

X_train = np.reshape(XX, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))

XX_test = np.reshape(X_test, (X_test.shape[0]*X_test.shape[1], X_test.shape[2]))
XX_test = scaler.transform(np.log(XX_test+1))
X_test = np.reshape(XX_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

In [22]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, BatchNormalization, Bidirectional, LayerNormalization
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import regularizers
import re
from keras.layers import Dropout

In [23]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    verbose=1,
    patience=10,
    mode='min',
    restore_best_weights=True)

In [24]:
METRICS =[
    keras.metrics.AUC(name ='accuracy'),
    keras.metrics.Recall(name ='recall'),
    keras.metrics.Precision(name = 'precision'),
]

In [25]:
n_steps = X_train.shape[1]
n_features = X_train.shape[2]

In [27]:
tf.keras.backend.clear_session()
tf.compat.v1.reset_default_graph()

from tensorflow.keras import initializers

model = Sequential()
model.add(tf.keras.layers.Masking(mask_value=0.,
                                   input_shape=(n_steps, n_features))) 
model.add(LSTM(300, return_sequences=False, 
               dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(1,activation='sigmoid'))
# try using category instead of binary
model.compile(loss = 'binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.01),metrics = METRICS)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 35, 20)            0         
                                                                 
 lstm (LSTM)                 (None, 300)               385200    
                                                                 
 dense (Dense)               (None, 1)                 301       
                                                                 
Total params: 385,501
Trainable params: 385,501
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
batch_size = 100
my_callbacks = [early_stopping]

neg = len(y_train[y_train ==0])
pos = len(y_train[y_train ==1])
total = neg + pos
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

history = model.fit(X_train, y_train, epochs = 100, batch_size=batch_size, 
                    validation_data=(X_test, y_test), 
                    callbacks = my_callbacks, class_weight=class_weight,
                    verbose = 2, shuffle=True)

Epoch 1/100
22/22 - 25s - loss: 0.9107 - accuracy: 0.6835 - recall: 0.6076 - precision: 0.5614 - val_loss: 0.4959 - val_accuracy: 0.8477 - val_recall: 0.6143 - val_precision: 0.8257 - 25s/epoch - 1s/step
Epoch 2/100
22/22 - 21s - loss: 0.6081 - accuracy: 0.7337 - recall: 0.5430 - precision: 0.7174 - val_loss: 0.6164 - val_accuracy: 0.8789 - val_recall: 0.7662 - val_precision: 0.6692 - 21s/epoch - 957ms/step
Epoch 3/100
22/22 - 19s - loss: 0.6124 - accuracy: 0.7145 - recall: 0.6430 - precision: 0.5486 - val_loss: 0.5471 - val_accuracy: 0.8801 - val_recall: 0.7372 - val_precision: 0.7013 - 19s/epoch - 877ms/step
Epoch 4/100
22/22 - 19s - loss: 0.5631 - accuracy: 0.7667 - recall: 0.6570 - precision: 0.6223 - val_loss: 0.6788 - val_accuracy: 0.8560 - val_recall: 1.0000 - val_precision: 0.3168 - 19s/epoch - 862ms/step
Epoch 5/100
22/22 - 19s - loss: 0.5392 - accuracy: 0.8041 - recall: 0.5747 - precision: 0.7116 - val_loss: 0.5475 - val_accuracy: 0.8780 - val_recall: 0.5290 - val_precision: 

In [29]:
results = model.evaluate(X_test, y_test)

58/58 [==============================] - 4s 77ms/step - loss: 0.3442 - accuracy: 0.9037 - recall: 0.6587 - precision: 0.8713


In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history, model):
    metrics = model.metrics_names
    plt.figure(figsize=(12, 8))
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        
        plt.subplot(3,2,n+1)
        plt.plot(history.epoch,  history.history[metric], color='b', label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
             color='r', linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        plt.legend()
    plt.show()

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_history(history, model)

%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

y_pred_train=model.predict_classes(X_train)
con_mat = tf.math.confusion_matrix(labels=y_train, predictions=y_pred_train).numpy()

cm_plot_labels = ['safe','risky']
cm = confusion_matrix(y_true=y_train, y_pred=y_pred_train)


plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='training Confusion Matrix')


y_pred_test=model.predict_classes(X_test)
con_mat = tf.math.confusion_matrix(labels=y_test, predictions=y_pred_test).numpy()

cm_plot_labels = ['safe','risky']
cm = confusion_matrix(y_true=y_test, y_pred=y_pred_test)


plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='test Confusion Matrix')